In [70]:
!pwd

/Users/alessandrorinaldi/code/alexanderdust/Dummy_DDP/Drowsiness detection


In [60]:
import os
import tensorflow
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils.np_utils import to_categorical
import random,shutil
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model


def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True,batch_size=1,target_size=(24,24),class_mode='categorical' ):

    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',class_mode=class_mode,target_size=target_size)

In [50]:
BS= 32
TS=(24,24)
train_batch= generator('backup_data/train',shuffle=True, batch_size=BS,target_size=TS)
valid_batch= generator('backup_data/valid',shuffle=True, batch_size=BS,target_size=TS)
SPE= len(train_batch.classes)//BS
VS = len(valid_batch.classes)//BS
print(SPE,VS)

Found 2467 images belonging to 4 classes.
Found 433 images belonging to 4 classes.
77 13


In [51]:
train_batch.classes[616]

0

In [52]:
img,labels= next(train_batch)
print(img.shape)

(32, 24, 24, 1)


In [53]:
# Model 
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
#32 convolution filters used each of size 3x3
#again
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),

#64 convolution filters used each of size 3x3
#choose the best features via pooling
    
#randomly turn neurons on and off to improve convergence
    Dropout(0.25),
#flatten since too many dimensions, we only want a classification output
    Flatten(),
#fully connected to get all relevant data
    Dense(128, activation='relu'),
#one more dropout for convergence' sake :) 
    Dropout(0.5),
#output a softmax to squash the matrix into output probabilities
    Dense(4, activation='softmax')
])

In [61]:
recall = tensorflow.keras.metrics.Recall(
    thresholds=None, top_k=None, class_id=None, name=None, dtype=None
)

In [62]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[recall])

In [63]:
model.fit_generator(train_batch, validation_data=valid_batch,epochs=15,steps_per_epoch=SPE ,validation_steps=VS)

Epoch 1/15
77/77 [==============================] - 8s 96ms/step - loss: 0.0949 - recall: 0.9665 - val_loss: 0.2114 - val_recall: 0.9608
Epoch 2/15
77/77 [==============================] - 7s 88ms/step - loss: 0.0819 - recall: 0.9593 - val_loss: 0.2078 - val_recall: 0.9621
Epoch 3/15
77/77 [==============================] - 7s 87ms/step - loss: 0.0611 - recall: 0.9625 - val_loss: 0.1911 - val_recall: 0.9638
Epoch 4/15
77/77 [==============================] - 7s 85ms/step - loss: 0.0524 - recall: 0.9641 - val_loss: 0.1717 - val_recall: 0.9665
Epoch 5/15
77/77 [==============================] - 6s 83ms/step - loss: 0.0579 - recall: 0.9671 - val_loss: 0.1506 - val_recall: 0.9689
Epoch 6/15
77/77 [==============================] - 6s 83ms/step - loss: 0.0497 - recall: 0.9691 - val_loss: 0.1588 - val_recall: 0.9703
Epoch 7/15
77/77 [==============================] - 6s 84ms/step - loss: 0.0383 - recall: 0.9706 - val_loss: 0.1891 - val_recall: 0.9706
Epoch 8/15
77/77 [=======================

In [67]:
y_pred = model.predict(valid_batch[0:10])

TypeError: '>=' not supported between instances of 'slice' and 'int'

In [ ]:
# Computing the probabilities of belonging to one of the 4 classes for the 10 000 images
predicted_probabilities = model.predict(X_test) # shape = 10_000 X 4

# Computing the predicted classes for all the images
predicted_classes = np.argmax(predicted_probabilities, axis = 1) # shape = 10_000 , 
​
# Confusion Matrix
​
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
​
​
cm = confusion_matrix(y_test, predicted_classes)
disp = ConfusionMatrixDisplay(confusion_matrix = cm,
                              display_labels = np.arange(0,4+1));
​
fig, ax = plt.subplots(figsize=(10,10));
disp.plot(cmap = "PuRd", ax = ax);

In [ ]:
model.save('models/cnnCat2.h5', overwrite=True)